# Mão na massa: Explorando os dados da Netflix

# Analisando dados da Netflix

Para começar, precisamos baixar o arquivo na plataforma da Kaggle:

 (https://www.kaggle.com/datasets/arnavsmayan/netflix-userbase-dataset), mas usando esse link:

 https://drive.google.com/drive/folders/1RuibSOdMp-cR6niAdbH8bRIj3PO4EOlh?usp=sharing

In [3]:
# importando os pacotes que iremos utilizar
import pandas as pd

# Coleta dos Dados

A primeira etapa, de coleta, envolve baixar e importar os dados da Netflix para o nosso ambiente.

Feito isso, utilizamos a função read_csv para importar o arquivo.

Nosso arquivo possui 10 colunas, e todas trazem informações sobre os usuários que utilizam a plataforma.

In [ ]:
# importando os dados da Netflix
df_netflix = pd.read_csv('dados/netflix.csv')
df_netflix

# Limpeza dos dados

Vamos começar a entender como nossos dados estão. Iremos explorar os tipos de dados que temos em cada coluna, e iremos checar dados nulos.

In [ ]:
df_netflix.info()

Com a função info() observamos que não temos dados nulos, vamos confirmar? 

In [ ]:
df_netflix.isnull().sum()

Agora, com os dados transformados, vamos analisá-los!

# Analisando os dados

Iniciando nossa análise exploratória, é importante entendermos os nossos dados de forma geral. Para isso, podemos rodar o comando describe() para trazer as estatísticas descritivas das colunas numéricas da nossa tabela.

**id_usuario**: essa coluna é uma coluna de identificador, então as estatísticas aqui mostradas não possuem valor para nós, vamos ignorá-las.

**receita_mensal:** a partir das estatísticas extraídas dessa coluna, podemos notar que, a média da receita mensal por usuário é cerca de $12.5.

**idade_usuario:** a idade média dos usuários que adquiriram a assinatura da netflix é de 38 anos, com a idade mínima de 26 anos e máxima de 51.

**país:** o país que teve mais assinatura foi os Estados Unidos, com 451 pessoas assinantes.



In [ ]:
df_netflix.describe().T

In [ ]:
df_netflix.describe(include='all').T

In [ ]:
# convertendo o tipo de dados para data
df_netflix['Last Payment Date'] = pd.to_datetime(df_netflix['Last Payment Date'])
df_netflix['Join Date'] = pd.to_datetime(df_netflix['Join Date'])

In [ ]:
df_netflix.info()

In [ ]:
# há quanto tempo as pessoas estão inscritas na Netflix?
df_netflix['duracao_assinatura'] = df_netflix['Last Payment Date'] - df_netflix['Join Date']
df_netflix['duracao_assinatura'] = df_netflix['duracao_assinatura'].dt.days
df_netflix

In [ ]:
# Com base na função describe() percebemos que a média de tempo que as pessoas se inscrevem é de cerca de 326 dias, quase um ano.
# Mas, pera aí, o que são esse números negativos no minimo?
df_netflix.describe().T

In [ ]:
# vamos validar esses numeros?
df_netflix[(df_netflix['duracao_assinatura'] < 0)]

In [16]:
13/2500

0.0052

In [ ]:
# Encontramos valores inconsistentes, onde o dado não faz sentido. Como não possuímos contexto suficiente sobre essa tabela para entender
# o motivo de termos dt_ultimo_pagamento maior que dt_assinatura, iremos retirar essas linhas para não enviezar nossos resultados.
df_netflix = df_netflix.drop(df_netflix[df_netflix['duracao_assinatura'] < 0].index)
df_netflix

In [ ]:
# Agora sim, tirando os dados negativos, nossa média de duração de assinatura subiu um pouco, e agora podemos visualizar que o tempo mínimo
# que as pessoas assinam a netflix é de 8 dias, e o máximo da base é 776 dias.
df_netflix.describe().T

In [ ]:
df_netflix.info()

In [ ]:
# Antes de continuarmos nossa análise, existe mais um passo que devemos fazer: resetar o index. Como retiramos algumas linhas da nossa tabela, o
# índice, ou seja, coluna mais à esquerda, porem ela ainda está com o intervalo de 1-2499.
df_netflix = df_netflix.reset_index(drop=True)
df_netflix

# agora sim! nossa coluna de index está coerente. Vamos continuar!

In [ ]:
# outra análise interessante de se fazer, será que existe um mês com um maior número de vendas?
df_netflix['dt_mes'] = df_netflix['Join Date'].dt.month
df_netflix['dt_mes'].value_counts()

In [ ]:
# Utilizando a função value_counts() conseguimos ter uma ideia de qual é a assinatura mais comum, porém, em números absolutos
# é dificil entender a proporção entre os 3 planos.
df_netflix["Subscription Type"].value_counts()

In [ ]:
# Para isso, colocamos um parâmetro dentro da nossa função, e ele nos devolve os valores em porcentagem sobre o total.
df_netflix["Subscription Type"].value_counts(normalize=True)

# daqui, percebemos que o pacote mais recorrente é o Basic.

In [ ]:
# Vamos entender também, quais sao os países com o maior número de assinaturas: em valor absoluto
df_netflix["Country"].value_counts()

In [ ]:
# Vamos entender também, quais sao os países com o maior número de assinaturas: em percentual
df_netflix["Country"].value_counts(normalize=True)

In [ ]:
# Vamos criar intervalos de idade para entender melhor nossos usuários?
# Iremos criar categorias de idades com o intervalo de 4 em 4 anos.
# ou seja, 20-24, 25-29, 30-34, 35-39, 40-44, 45-49, 50-54, 55-59, +60

df_netflix.loc[df_netflix['Age'] < 25, 'grupo_idade'] = '20-24'
df_netflix.loc[(df_netflix['Age'] >= 25) & (df_netflix['Age'] < 30), 'grupo_idade'] = '25-29'
df_netflix.loc[(df_netflix['Age'] >= 30) & (df_netflix['Age'] < 35), 'grupo_idade'] = '30-34'
df_netflix.loc[(df_netflix['Age'] >= 35) & (df_netflix['Age'] < 40), 'grupo_idade'] = '35-39'
df_netflix.loc[(df_netflix['Age'] >= 40) & (df_netflix['Age'] < 45), 'grupo_idade'] = '40-44'
df_netflix.loc[(df_netflix['Age'] >= 45) & (df_netflix['Age'] < 50), 'grupo_idade'] = '45-49'
df_netflix.loc[(df_netflix['Age'] >= 50) & (df_netflix['Age'] < 55), 'grupo_idade'] = '50-54'
df_netflix.loc[(df_netflix['Age'] >= 55) & (df_netflix['Age'] < 60), 'grupo_idade'] = '55-59'
df_netflix.loc[df_netflix['Age'] >= 60, 'grupo_idade'] = '60+'
df_netflix

In [ ]:
# com o grupo de idades feito, qual será a faixa etária mais recorrente?
df_netflix["grupo_idade"].value_counts()

# Visualização dos dados

In [ ]:
# !pip install python-matplotlib
# !pip install seaborn

In [41]:
# importando os pacotes que iremos utilizar
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# definir tamanho da figura
plt.figure(figsize=(8, 6))

# instanciar o tipo de plotagem, no caso, um countplot usando x como as faixas etárias
sns.countplot(data = df_netflix,
              x = 'grupo_idade')

# definir texto do título e eixos x e y
plt.title('Contagem de faixas etárias')
plt.xlabel('Faixa etária')
plt.ylabel('Contagem')
plt.show()

In [ ]:
# definir tamanho da figura
plt.figure(figsize=(8, 6))

# instanciar o tipo de plotagem, no caso, um countplot usando x como os tipos de plano
sns.countplot(data = df_netflix,
              x = 'Subscription Type')

# definir texto do título e eixos x e y
plt.title('Contagem dos tipos de plano de assinatura')
plt.xlabel('Plano de assinatura')
plt.ylabel('Contagem')

plt.show()

In [ ]:
# definir tamanho da figura
plt.figure(figsize=(10, 6))

# instanciar o tipo de plotagem, no caso, um countplot usando x como os tipos
# de plano com tonalidades das barras definidas pela faixa etária

sns.countplot(data = df_netflix,
              x = 'Subscription Type',
              hue = 'grupo_idade')

# definir texto do título e eixos x e y

plt.title('Planos de assinatura por faixa etária')
plt.xlabel('Faixa etária')
plt.ylabel('Contagem')

# plotar e definir texto da legenda
plt.legend(title = 'Faixa etária')

plt.show()

In [ ]:
# definir tamanho da figura
plt.figure(figsize=(12, 6))

# instanciar o tipo de plotagem, no caso, um countplot usando x como os países
# com tonalidades das barras definidas pela faixa etária

sns.countplot(data = df_netflix,
              x = 'Country',
              hue = 'grupo_idade')

# definir texto do título e eixos x e y

plt.title('Países por faixa etária')
plt.xlabel('País')
plt.ylabel('Contagem')

# como o texto do eixo x é longo, podemos rotacioná-lo para melhorar a visualização
plt.xticks(rotation=330)

# plotar e definir texto da legenda
plt.legend(title = 'Faixa etária')
plt.show()

In [ ]:
# definir tamanho da figura

plt.figure(figsize=(12, 6))

# instanciar o tipo de plotagem, no caso, um countplot usando x como os países
# com tonalidades das barras definidas pelo tipo de plano

sns.countplot(data = df_netflix,
              x = 'Country',
              hue = 'Subscription Type')

# definir texto do título e eixos x e y
plt.title('Países por tipo de plano')
plt.xlabel('País')
plt.ylabel('Contagem')

# como o texto do eixo x é longo, podemos rotacioná-lo para melhorar a visualização
plt.xticks(rotation=330)

# plotar e definir texto da legenda
plt.legend(title = 'Tipo de plano')

plt.show()

# Hora de praticar

Agora é sua vez de colocar a mão na massa! Em grupos, respondam as seguintes perguntas sobre os dados:

1. Quantos usuários utilizam a Netflix pelo Tablet?
2. Qual é a representatividade (em %) do sistema operacional mais utilizado para assistir Netflix?
3. Crie visualizações que responda as seguintes perguntas:
    1. Qual é o sistema operacional mais usado para cada faixa etária? Existe algum padrão? O tipo de aparelho muda de acordo com o grupo de idade?
    2. No Brasil, qual é o sistema operacional mais utilizado?